In [ ]:
import sqlite3
import openai
import random

# Set OpenAI API key
openai.api_key = "your_openai_api_key"

# ==================================================
# OpenAI GPT-4 Query Function
# ==================================================
def query_llm(prompt):
    """Queries OpenAI GPT-4 for claim processing."""
    response = client.ChatCompletion.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are an expert insurance claims processor."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=100
    )
    return response['choices'][0]['message']['content'].strip()

# ==================================================
# AI Agents
# ==================================================
class ReactiveAgent:
    """Processes claims immediately without memory or learning."""
    def process_claim(self, claim_text):
        category = query_llm(f"Categorize this insurance claim: {claim_text}")
        return f"Categorized as {category}"

class ReflectionAgent:
    """Improves claim categorization over time using memory."""
    def __init__(self):
        self.memory = {}

    def process_claim(self, claim_text):
        if claim_text in self.memory:
            return f"Using learned category: {self.memory[claim_text]}"
        category = query_llm(f"Categorize this insurance claim: {claim_text}")
        self.memory[claim_text] = category
        return f"Categorized as {category}"

class ReActAgent:
    """Reasons before acting on a claim."""
    def reason(self, claim_text):
        return query_llm(f"Analyze this claim before making a decision: {claim_text}")

    def act(self, claim_text):
        return query_llm(f"Now process this claim based on your reasoning: {claim_text}")

    def process_claim(self, claim_text):
        return f"{self.reason(claim_text)}\n{self.act(claim_text)}"

class ReWOOAgent:
    """Decides whether to observe data or use prior knowledge."""
    def observe(self, claim_text):
        return query_llm(f"Retrieve external fraud data for this claim: {claim_text}")

    def reason(self, claim_text):
        return query_llm(f"Reason internally about this claim without external data: {claim_text}")

    def process_claim(self, claim_text):
        return self.observe(claim_text) if random.choice([True, False]) else self.reason(claim_text)

class DeliberativeAgent:
    """Plans before making a claim decision."""
    def plan(self, claim_text):
        return query_llm(f"Plan a step-by-step process to assess this claim: {claim_text}")

    def process_claim(self, claim_text):
        return f"Plan: {self.plan(claim_text)}\nFinal Decision: {query_llm(f'Final decision for this claim: {claim_text}')}
"

class LearningAgent:
    """Learns fraud patterns over time."""
    def __init__(self):
        self.fraud_memory = {}

    def detect_fraud(self, claim_text):
        if claim_text in self.fraud_memory:
            return f"Using learned fraud pattern: {self.fraud_memory[claim_text]}"
        fraud_status = query_llm(f"Analyze if this claim is fraudulent: {claim_text}")
        self.fraud_memory[claim_text] = fraud_status
        return f"Fraud Analysis: {fraud_status}"

class HybridAgent:
    """Combines memory, reasoning, learning, and planning."""
    def __init__(self):
        self.memory = {}

    def learn(self, claim_text, category):
        self.memory[claim_text] = category

    def reflect(self, claim_text):
        return self.memory.get(claim_text, "Unknown, needs processing.")

    def process_claim(self, claim_text):
        if claim_text in self.memory:
            return self.reflect(claim_text)
        category = query_llm(f"Categorize this claim: {claim_text}")
        self.learn(claim_text, category)
        return f"Planned Process -> Learned: {category}"

# ==================================================
# Database Setup
# ==================================================
def initialize_database():
    """Creates the database and claims table if not exists."""
    conn = sqlite3.connect("insurance_claims.db")
    cursor = conn.cursor()
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS claims (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            claim_text TEXT,
            category TEXT,
            fraud_status TEXT,
            risk_level TEXT,
            payout_estimate TEXT,
            policy_validation TEXT,
            final_decision TEXT
        )
    """)
    conn.commit()
    conn.close()

initialize_database()

# ==================================================
# Database Operations
# ==================================================
def save_claim_to_db(claim_text, category, fraud_status, risk_level, payout_estimate, policy_validation, final_decision):
    """Stores a processed claim in the database."""
    conn = sqlite3.connect("insurance_claims.db")
    cursor = conn.cursor()
    cursor.execute("""
        INSERT INTO claims (claim_text, category, fraud_status, risk_level, payout_estimate, policy_validation, final_decision)
        VALUES (?, ?, ?, ?, ?, ?, ?)
    """, (claim_text, category, fraud_status, risk_level, payout_estimate, policy_validation, final_decision))
    conn.commit()
    conn.close()

def fetch_claim_from_db(claim_text):
    """Retrieves a claim from the database."""
    conn = sqlite3.connect("insurance_claims.db")
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM claims WHERE claim_text = ?", (claim_text,))
    result = cursor.fetchone()
    conn.close()
    return result

# ==================================================
# Agent Management and Processing
# ==================================================
agents = {
    "Reactive": ReactiveAgent(),
    "Reflection": ReflectionAgent(),
    "ReAct": ReActAgent(),
    "ReWOO": ReWOOAgent(),
    "Deliberative": DeliberativeAgent(),
    "Learning": LearningAgent(),
    "Hybrid": HybridAgent()
}

def process_user_query(user_input, agent_type):
    """Processes a user query using the selected AI agent."""
    if agent_type not in agents:
        print("Invalid agent type.")
        return

    agent = agents[agent_type]
    previous_claim = fetch_claim_from_db(user_input)

    if previous_claim:
        print("\n🔹 Retrieved Claim from Database:")
        print(f"Category: {previous_claim[2]}, Fraud Status: {previous_claim[3]}, Risk Level: {previous_claim[4]}")
        print(f"Payout Estimate: {previous_claim[5]}, Policy Validation: {previous_claim[6]}, Final Decision: {previous_claim[7]}")
        return

    print(f"\n🔹 Processing with {agent_type} Agent:")
    result = agent.process_claim(user_input)
    print(result)

# ==================================================
# User Interaction Loop
# ==================================================
while True:
    user_input = input("\n🔹 Enter a claim description or type 'exit' to quit: ")
    if user_input.lower() == "exit":
        break
    agent_type = input("Select agent type (Reactive, Reflection, ReAct, ReWOO, Deliberative, Learning, Hybrid): ")
    process_user_query(user_input, agent_type)


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langgraph.graph import Graph

# Initialize LLM
llm = ChatOpenAI(model="gpt-4", openai_api_key="your_api_key")

# Define claim categorization function
def claim_categorization(claim):
    """Categorization Agent - Classifies the insurance claim type."""
    response = llm.predict(f"Classify this insurance claim: {claim}")
    return response

# Define fraud detection function
def fraud_detection(claim_category):
    """Fraud Detection Agent - Checks if the claim is fraudulent."""
    response = llm.predict(f"Check if this claim is fraudulent: {claim_category}")
    return response

# Define tools for agents
categorization_tool = Tool(
    name="Claim Categorization",
    func=claim_categorization,
    description="Classifies an insurance claim into categories."
)

fraud_detection_tool = Tool(
    name="Fraud Detection",
    func=fraud_detection,
    description="Analyzes classified claims and checks for fraud."
)

# Create agents
categorization_agent = initialize_agent(
    tools=[categorization_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

fraud_detection_agent = initialize_agent(
    tools=[fraud_detection_tool],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Create Multi-RAG AI workflow
graph = Graph()

# Add nodes with agents
graph.add_node("categorization_agent", categorization_agent.run)
graph.add_node("fraud_detection_agent", fraud_detection_agent.run)

# Define execution flow
graph.add_edge("categorization_agent", "fraud_detection_agent")

# Compile the workflow
workflow = graph.compile()

# Execute workflow
result = workflow.invoke("Car accident involving minor damage.")
print(result)
